In [4]:
# Import Necessary Packages
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_val_score,
    cross_validate,
    train_test_split,
)
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline

import os
import sys
sys.path.append(os.path.join(os.path.abspath(".."), (".."), "code"))
#from plotting_functions import *
#from utils import *

import altair as alt
from vega_datasets import data
# Simplify working with large datasets in Altair
alt.data_transformers.enable('vegafusion')

pd.set_option("display.max_colwidth", 200)

In [5]:
df = pd.read_csv("data/cleaned_hm.csv", index_col=0)
sample_df = df.dropna()
sample_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/cleaned_hm.csv'

In [1]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=123)

vec = CountVectorizer()
X_train_enc = vec.fit_transform(X_train)

lr = LogisticRegression(max_iter=2000)

param_dist = {
    "C": loguniform(1e-3, 1e3)
}

random_search = RandomizedSearchCV(
    lr,
    param_distributions=param_dist,
    n_iter=50,
    n_jobs=-1,
    return_train_score=True
)

random_search.fit(X_train_enc, y_train)

NameError: name 'train_test_split' is not defined

In [2]:
pipe = make_pipeline(CountVectorizer(), LogisticRegression(max_iter=2000))

random_search = RandomizedSearchCV(
    pipe,
    param_distributions={"logisticregression__C": loguniform(1e-3, 1e3)},
    n_iter=50,
    n_jobs=-1,
    return_train_score=True
)

random_search.fit(X_train, y_train)

NameError: name 'make_pipeline' is not defined

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=123) 

pipe_lr = make_pipeline(CountVectorizer(), LogisticRegression(max_iter=2000)) 

pipe_lr.fit(X_train, y_train) 
vocab = pipe_lr.named_steps['countvectorizer'].get_feature_names_out()

param_dist = { 
    "logisticregression__C": loguniform(1e-3, 1e3), 
    "countvectorizer__max_features": randint(100, len(vocab)) 
} 

random_search = RandomizedSearchCV( 
    pipe_lr, 
    param_distributions=param_dist, 
    n_iter=50, 
    n_jobs=-1, 
    return_train_score=True 
)

random_search.fit(X_train, y_train)

NameError: name 'train_test_split' is not defined